# 경로 지정

In [3]:
# 확인할 행 번호
TARGET_ROW = 1084

In [4]:
import pandas as pd
from pathlib import Path
import shutil  # 파일 복사용

# -------------------------------------------------------
# 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")
CSV_PATH = BASE_DIR / "metadata_yolo_final.csv"

# -------------------------------------------------------
# CSV 로드 및 행 접근
# -------------------------------------------------------
meta = pd.read_csv(CSV_PATH)

# -------------------------------------------------------
# 개별 경로 생성
# -------------------------------------------------------
FRAME_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['frame_path'])
KEYPOINTS_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['keypoints_path'])
INTERP_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['interp_json_path'])

# -------------------------------------------------------
# 존재 여부 확인 및 폴더 생성
# -------------------------------------------------------
for p in [FRAME_DIR, KEYPOINTS_DIR, INTERP_DIR]:
    if not p.exists():
        if p == INTERP_DIR:
            p.mkdir(parents=True, exist_ok=True)
            print(f"📁 생성 완료: {p}")
        else:
            print(f"❌ 없음: {p}")
    else:
        print(f"✅ 존재: {p}")

print(f"\nis_train = {meta.loc[TARGET_ROW]['is_train']}")
print(f"🎯 현재 처리 대상: {Path(FRAME_DIR).name}")


✅ 존재: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/1_FRAME/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기
✅ 존재: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기
📁 생성 완료: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기

is_train = True
🎯 현재 처리 대상: 경사진 침대에서 몸통 돌리기


# 함수

## Frame 시각화 함수

## 선형 보간 함수

## IQR 시각화 함수

## Velocity 시각화 함수

## smoothing 시각화 함수

## 영상 생성 함수

In [ ]:
import cv2
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ===========================================================
# 단일 Overlay 생성 함수
# ===========================================================
def create_overlay(frame_dir: str, json_dir: str, out_mp4: str, fps: int = 30,
                   kp_radius: int = 4, line_thickness: int = 2):
    """
    프레임 + keypoints JSON → overlay mp4 생성 (COCO 17kp 구조)
    - 좌우 반전 없음
    - 0~4번 keypoints 제외 (얼굴 제외)
    - L/R 색상 구분
    - 하단 안내문구 표시 ("L: Blue | R: Red")
    """

    frame_files = sorted(Path(frame_dir).glob("*.jpg"))
    if not frame_files:
        print(f"[WARN] No frames found in {frame_dir}")
        return

    out_mp4 = Path(out_mp4)
    out_mp4.parent.mkdir(parents=True, exist_ok=True)

    # 해상도 확인
    sample = cv2.imread(str(frame_files[0]))
    h, w = sample.shape[:2]
    writer = cv2.VideoWriter(str(out_mp4), cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # 색상 설정
    COLOR_SK = (50, 50, 50)   # Skeleton = 짙은 회색
    COLOR_L  = (255, 0, 0)    # 왼쪽 keypoints = 파랑
    COLOR_R  = (0, 0, 255)    # 오른쪽 keypoints = 빨강
    COLOR_NEUTRAL = (0, 255, 0) # 중앙부 keypoints = 초록

    LEFT_POINTS  = [5,7,9,11,13,15]
    RIGHT_POINTS = [6,8,10,12,14,16]

    for frame_path in tqdm(frame_files, total=len(frame_files), desc=f"{Path(frame_dir).name}", unit="frame"):
        frame = cv2.imread(str(frame_path))
        json_path = Path(json_dir) / (frame_path.stem + ".json")

        if json_path.exists():
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "instance_info" in data and len(data["instance_info"]) > 0:
                inst = data["instance_info"][0]
                kpts = np.array(inst["keypoints"])
                skeleton = data.get("meta_info", {}).get("skeleton_links", [])

                # Skeleton
                for i, j in skeleton:
                    if i < len(kpts) and j < len(kpts):
                        if i < 5 or j < 5:
                            continue
                        pt1, pt2 = tuple(map(int, kpts[i])), tuple(map(int, kpts[j]))
                        cv2.line(frame, pt1, pt2, COLOR_SK, line_thickness)

                # Keypoints
                for idx, (x, y) in enumerate(kpts):
                    if idx < 5 or x <= 0 or y <= 0:
                        continue
                    if idx in LEFT_POINTS:
                        color = COLOR_L
                    elif idx in RIGHT_POINTS:
                        color = COLOR_R
                    else:
                        color = COLOR_NEUTRAL
                    cv2.circle(frame, (int(x), int(y)), kp_radius, color, -1)

                # 안내 문구
                legend_text = "L: Blue   |   R: Red"
                cv2.putText(frame, legend_text, (20, h - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

        frame = cv2.flip(frame, 1)  # 좌우 반전 복원
        writer.write(frame)

    writer.release()
    print(f"✅ Overlay 완료 → {out_mp4}")
